In [1]:
import dash
from dash.dependencies import Input, Output
import dash_bio as dashbio
from dash import html, dcc
import json

In [2]:
# Python functions for data wrangling

# read list of bands files available in the online database
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup

def read_url(url):
    bands = {}   
    url = url.replace(" ","%20")
    req = Request(url)
    a = urlopen(req).read()
    soup = BeautifulSoup(a, 'html.parser')
    x = (soup.find_all('a'))
    for i in x:
        file_name = i.extract().get_text()
        if file_name.endswith('.json'):
            bands[file_name.replace('.json', '')] = ''
    # read labels from the list.txt (if it exists)
    try:
        for line in urlopen(url+"/list.txt"):
            pair = line.decode('utf-8').strip().split(',')
            if pair[0] in bands.keys():
                bands[pair[0]] = pair[1]
    except:
        print('')            
            
    return(bands)


# derive list of available chromosomes
import pandas as pd
import re

p = re.compile(r'(\d+)')

def extract_num(s, p, ret=0):
    search = p.search(s)
    if search:
        return int(search.groups()[0])
    else:
        return ret

def get_chromosomes(bands_file_url):
    chromosomes = ''
    try:
        data = json.loads(urlopen(bands_file_url).read())
        df = pd.DataFrame.from_dict(data['chrBands'])
        ch = set(df.iloc[:,0].str.split(' ', expand=True)[0].to_list())
        chromosomes = sorted(list(ch), key=lambda s: extract_num(s, p, float('inf')))
    except:
        print('Error')
    return chromosomes


In [3]:
# Execute functions to provide initial settings

default_url = "https://aedawid.github.io/ideogram/database/bands/"
bands = read_url(default_url)
default_org = list(bands.keys())[0] + ".json"
chromosomes = get_chromosomes(default_url+default_org)
annotations = list(read_url(default_url.replace('bands','annotations')).keys())
annotations.insert(0,'None')

In [4]:
# CSS styles

css_btn = {'font-size':'20px', 'background-color':'#008CBA', 'color':'white', 'border-radius':'8px', 'border':'1px solid #006B88', 'marginBottom':'10px'}
css_div = {'display':'inline-block'}
css_lab = {'color':'#008CBA', 'font-size':'16px', 'font-style':'italic', 'display':'inline-block'}
css_inp = {'marginBottom':'10px', 'width':'60%', 'display':'inline-block', 'font-size':'14px', 'padding':'6px 0'}
css_val = {'width':'30%', 'marginBottom':'20px'}
css_rad = {'padding': '1vh 2.5vw 0 0'}

# Application layout

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Button('Show Options', id='options', n_clicks=0, style=css_btn),
  html.Div([
    html.Div([
        
        html.Label('Provide URL to online bands:', style={**css_lab, 'width':'65%'}),
        html.Label('Select bands data file:', style={**css_lab, 'width':'35%'}),
        dcc.Input(id="input-url", type="text",
            value="https://aedawid.github.io/ideogram/database/bands/", 
            style=css_inp),
      html.Div([
        dcc.Dropdown(id='dash-bands', value=list(bands.keys())[0], multi=False,
            options=[{'label': str(bands[i]), 'value': str(i)} for i in bands],
            style={'display':'block', 'width':'100%', 'verticalAlign':'top'}),
      ], style={'display':'inline-block', 'width':'36%', 'marginLeft':'3%'}),
        
      html.Label('Select chromosomes to display on the ideogram:', style={**css_lab, 'width':'100%'}),
      dcc.Dropdown(
        id='dash-chromosomes',
        options=[{'label': str(i), 'value': str(i)} for i in chromosomes],
        multi=True,
        value=chromosomes
      ),
        
      html.Label('Provide URL to online annotations:', style={**css_lab, 'width':'65%', 'marginTop':'10px'}),
      html.Label('Select annotations data file:', style={**css_lab, 'width':'35%'}),
      dcc.Input(id="annot-url", type="text", 
              value="https://aedawid.github.io/ideogram/database/annotations/human_10.json", 
              style={**css_inp, 'verticalAlign':'top'}),
      html.Div([
        dcc.Dropdown(id='dash-annots', value='None', multi=False,
          options=[{'label': str(i), 'value': str(i)} for i in annotations]),
      ], style={'display':'inline-block', 'width':'36%', 'marginLeft':'3%'}),
        
    ], id='data-opts', style={**css_div, 'width':'55%'}),
      
    html.Div([
      html.Label('Rotable:', style={**css_lab, **css_val}),
      dcc.RadioItems(id='rotatable', options=['YES', 'NO'], value='YES', style={**css_div, 'width':'70%'}, labelStyle=css_rad),
      html.Label('Orientation:', style={**css_lab, **css_val}),
      dcc.RadioItems(id='orientation', options=['vertical', 'horizontal'], value='vertical', style={**css_div, 'width':'70%'}, labelStyle=css_rad),
      html.Label('Dimensions:', style={**css_lab, **css_val}),
      dcc.Input(id="chr-height", type="number", value=600, 
          style={'marginRight':'2%', 'width':'20%', 'display':'inline-block'}),
      dcc.Input(id="chr-width", type="number", value=20, 
          style={'marginRight':'2%', 'width':'20%', 'display':'inline-block'}),
      dcc.Input(id="chr-margin", type="number", value=10, 
        style={'marginBottom':'10px', 'width':'20%', 'display':'inline-block'}),
      html.Label('Genomic range:', style={**css_lab, **css_val}),
      dcc.Input(id="brush", type="text", placeholder="e.g., chr1:104325484-119977655",
          style={'marginRight':'2%', 'width':'66%', 'display':'inline-block'}),
    ], id='styling-opts', style={**css_div, 'width':'40%','marginLeft':'5%', 'verticalAlign':'top'}),
  ], id='optionsDiv'), 

  dashbio.Ideogram(id='dashbio-ideogram',),
])


In [ ]:
# Javascript clientside callbacks

app.clientside_callback(
    """
    function(largeValue1, largeValue2) {
        var x = document.getElementById("optionsDiv");
        var y = document.getElementById("options");
        if (x.style.display === "none") {
            x.style.display = "block";
            y.style.backgroundColor = "#D6F2FA";
            y.style.color = "#90B6C1";
            y.innerText = "Hide Options";
        } else {
            x.style.display = "none";
            y.style.backgroundColor = "#008CBA";
            y.style.color = "white";
            y.innerText = "Show Options";
        }
    }
    """,
    Output('optionsDiv', 'style'),
    Input('options', 'n_clicks'),
)


# Callbacks responsive to changes in Dash widgets (options panel)

@app.callback(
    [Output('dash-bands', 'options'), Output('dash-bands', 'value')],
    Input('input-url', 'value')
)
def update_bands_options(value):
    bands = read_url(value)
    return [bands, list(bands.keys())[0]]

@app.callback(
    [Output('dash-chromosomes', 'options'), Output('dash-chromosomes', 'value')],
    [Input('input-url', 'value'), Input('dash-bands', 'value')]
)
def update_chromosomes_options(url, band):
    if not band.endswith('.json'):
        band += ".json"
    chromosomes = get_chromosomes(url+band)
    return [chromosomes, chromosomes]

@app.callback(
    [Output('dash-annots', 'options'), Output('dash-annots', 'value')],
    Input('input-url', 'value')
)
def update_annotations_options(url):
    annotations = list(read_url(url.replace('bands','annotations')).keys())
    annotations.insert(0,'None')
    return [annotations, 'None']

@app.callback(
    Output('annot-url', 'value'),
    [Input('input-url', 'value'), Input('dash-annots', 'value')]
)
def update_annotations(url, filename):
    if filename != 'None':
        annotations = url.replace('bands','annotations') + filename + ".json"
    else:
        annotations = ''
    return annotations


# Callbacks that directly change ideogram

@app.callback(
    Output('dashbio-ideogram', 'dataDir'),
    Input('input-url', 'value')
)
def update_dataDir(value):
    return value

@app.callback(
    Output('dashbio-ideogram', 'organism'),
    Input('dash-bands', 'value')
)
def update_organism(value):
    return value

@app.callback(
    Output('dashbio-ideogram', 'chromosomes'),
    Input('dash-chromosomes', 'value')
)
def update_chromosomes(value):
    return value

@app.callback(
    Output('dashbio-ideogram', 'annotationsPath'),
    Input('annot-url', 'value')
)
def update_annotations(value):
    return value

@app.callback(
    Output('dashbio-ideogram', 'brush'),
    Input('brush', 'value')
)
def update_genomic_range(value):
    return value

@app.callback(
    Output('dashbio-ideogram', 'rotatable'),
    Input('rotatable', 'value')
)
def update_rotatable(value):
    if value == 'YES':
        return True
    else:
        return False

@app.callback(
    Output('dashbio-ideogram', 'orientation'),
    Input('orientation', 'value')
)
def update_orientation(value):
    return value

@app.callback(
    [Output('dashbio-ideogram', 'chrHeight'), Output('dashbio-ideogram', 'chrWidth'), Output('dashbio-ideogram', 'chrMargin')],
    [Input('chr-height', 'value'), Input('chr-width', 'value'), Input('chr-margin', 'value')]
)
def update_chromosome_size(height, width, margin):
    return [height, width, margin]

app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Sep/2022 09:20:21] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:20:21] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:20:21] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Sep/2022 09:20:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:20:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:20:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:20:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:20:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:20:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:20:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:20:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22

127.0.0.1 - - [22/Sep/2022 09:20:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:20:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:22:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:22:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:22:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:22:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:22:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:23:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2022 09:23:02] "POST /_dash-update-component HTTP/1.1" 200 -
